In [1]:
import syft as sy
duet = sy.launch_duet(loopback=True)

# Notes: Review layers of model for possible improvement, add tags to iformation . Add notes

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 


[2021-10-10T13:23:39.239638-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.datasets.Omniglot not supported in 0.8.0a0
[2021-10-10T13:23:39.241980-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.datasets.Omniglot.__len__ not supported in 0.8.0a0
[2021-10-10T13:23:39.247332-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.datasets.Places365 not supported in 0.8.0a0
[2021-10-10T13:23:39.251732-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.datasets.Places365.__len__ not supported in 0.8.0a0
[2021-10-10T13:23:39.256605-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.transforms.GaussianBlur not supported in 0.8.0a0
[2021-10-10T13:23:39.263271-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.transforms.ConvertImageDtype not supported in 0.8.0a0
[2021-10-10T13:23:39.271918-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.transforms.functional.adjust_sharpness not supported in 

♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


/Users/robertbrown/opt/anaconda3/envs/farm/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/Users/robertbrown/opt/anaconda3/envs/farm/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore
[2021-10-10T13:23:49.909652-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.datasets.Omniglot not supported in 0.8.0a0
[2021-10-10T13:23:49.910656-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.datasets.Omniglot.__len__ not supported in 0.8.0a0
[2021-10-10T13:23:49.911413-0400][CRITICAL][logger]][12628] Skipping torchvision.torchvision.datasets


♫♫♫ > CONNECTED!



In [3]:
duet.requests.add_handler(action="accept")


In [4]:
duet.store

[]

In [5]:
import os 
import pandas as pd
import numpy as np
from sklearn import datasets
import torch as th
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

os.chdir('data')

In [7]:
#Data Pre-Processing

df = pd.read_csv("wheat-2013-supervised.csv")
df1 = pd.read_csv("wheat-2014-supervised.csv")
df = pd.concat([df, df1],axis=0)

In [9]:
conditions = [
    (df['Yield'] <= 31),
    (df['Yield'] > 31)
    ]

# create a list of the values we want to assign for each condition
values = [0,1]

# create a new column and use np.select to assign values to it using our lists as arguments
df['Yield_Cat'] = np.select(conditions, values)


In [10]:
df['EstDistEquator'] = df['Latitude'] * 69

data_new = df[['EstDistEquator','precipTypeIsOther','temperatureMax','NDVI','visibility','dewPoint','pressure','humidity','cloudCover','windSpeed', 'Yield_Cat' ]]

data_new = data_new.dropna(axis=0)

y = data_new[['Yield_Cat']]
x = data_new.drop(['Yield_Cat'],axis=1)

In [12]:
# Normalize feature data
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
    
])

In [13]:
#Convert data and labels to tensors

x = num_pipeline.fit_transform(x)

x = th.FloatTensor(x)

y = y.to_numpy()

y = th.FloatTensor(y).reshape(-1)

In [14]:
# Labels vector shape
print(y.shape)

torch.Size([359427])


In [15]:
# Provide information about data to be sent to the Data Scientist 
x = x.tag("Farm sensor data")
y = y.tag("Wheat yield labels")

x = x.describe(
    "This is a dataset for wheat yield of 359,427 examples using 10 features w/ IoT sensor data."
)
y = y.describe("Labels for wheat yield. 0 = low yield. 1 = high yield")

In [16]:
#Send pointers to the data to the Data Scientist 

data_pointer = x.send(duet, pointable=True)
target_pointer = y.send(duet, pointable=True)

In [17]:
#Display the sent information (Pointers to tensor objects)
duet.store

[<syft.proxy.torch.TensorPointer object at 0x7fa7e3a3eb80>, <syft.proxy.torch.TensorPointer object at 0x7fa7ceffb1f0>]

In [18]:
# For testing purposes, accept all requests from Data Scientist 
duet.requests.add_handler(action="accept")